# Aula 5: funçoes e estrutura de dados

In [1]:
import pandas as pd

In [21]:
# load dataset
data = pd.read_csv( 'C:/Users/PICHAU/Desktop/AnaliseDeDados/DataAnalysis/datasets/kc_house_data.csv' )
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [22]:
# data dimensions
print( 'Number of rows: {}'.format( data.shape[0] ) )
print( 'Number of columns: {}'.format( data.shape[1] ) )

Number of rows: 21613
Number of columns: 21


In [23]:
# convert object to date
data['date'] = pd.to_datetime( data['date'] )

In [ ]:
# showing types
data.dtypes

In [32]:
# Descriptive statistics
# essa analise descritiva é muito boa para olharmos de primeira como que funciona o nosso conjunto de dados
num_attributes = data.select_dtypes( include=['int64','float64'] )


# central tendency - media, mediana
import numpy as np
media =  pd.DataFrame( num_attributes.apply( np.mean, axis=0 ) )
#apply aplica uma funçao em todas as linhas ou colunas,
#axis=0 é para aplicar ao longo das linhas
mediana = pd.DataFrame( num_attributes.apply( np.median, axis=0 ) )


# dispersion - std,min, max
std =  pd.DataFrame( num_attributes.apply( np.std, axis=0 ) )
min_ = pd.DataFrame( num_attributes.apply( np.min, axis=0 ) )
max_ = pd.DataFrame( num_attributes.apply( np.min,axis=0 ) )

df1  = pd.concat( [max_,max_,media,mediana,std], axis=1 ).reset_index()
#concatena o que vc passar, no caso vamos passar uma sequencia de dataframes
df1.columns = ['atributes', 'maximo' ,'minimo', 'media', 'mediana', 'std']
# desse jeito tambem tem como nos renomearmos as colunas
pd.set_option('display.float_format', lambda x: '%.3f' % x)
#tira notacao cientifica

print( 'Number of rows: {}'.format( df1.shape[0] ) )
print( 'Number of columns: {}'.format( df1.shape[1] ) )


Number of rows: 20
Number of columns: 6


In [36]:
# Crie uma nova coluna chamada: "dormitory_type"
# "bedrooms" == 1 => 'studio'
# "bedrooms" == 2 => 'apartament'
# "bedrooms" > 2 => 'house'

data['dormitory_type'] = 'NA'

# usando for:
for i in range( len(data) ):# olhar como o range funciona, é bem simples
    if data.loc[i, 'bedrooms'] == 1:
        data.loc[i,'dormitory_type'] = 'studio'
    elif data.loc[i, 'bedrooms'] == 2:
        data.loc[i,'dormitory_type'] = 'apartament'
    else:
        data.loc[i,'dormitory_type'] = 'house'


In [38]:
# Quantidade de imoveis por nivel?
#     - nivel 0 : Preço entre 0 e 321.950
#     - nivel 1 : Preço entre 321.950 e 450.000
#     - nivel 2 : Preço entre 450.000 e 645.000
#     - nivel 3 : Preço acima de 645.000

data['level'] = None
for i in range( len(data) ):
    if data.loc[i, 'price'] <= 321950:
        data.loc[i, 'level'] = 0
    elif ( data.loc[i, 'price'] > 321950 ) & ( data.loc[i, 'price'] <= 450000 ):
        data.loc[i, 'level'] = 1
    elif ( data.loc[i, 'price'] > 450000 ) & ( data.loc[i, 'price'] <= 645000 ):
        data.loc[i, 'level'] = 2
    else:
        data.loc[i, 'level'] = 3



 # Pegando informaçoes do endereço das casas

In [47]:
from geopy.geocoders import Nominatim

# Initialize API

geolocator = Nominatim( user_agent='geoapiExercises' )

# create empty
data.loc[:, 'house_number'] = 'NA'
data.loc[:, 'road'] = 'NA'

# processing data
for i in range( len(data) ):
    print( 'Loop: {}/{}'.format( i, len( data ) ) )

    query = str( data.loc[i, 'lat'] ) + ',' + str( data.loc[i, 'long'] )
    response = geolocator.reverse( query )

    if 'house_number' in response.raw['address']['house_number']:
        data.loc[i, 'house_number'] = response.raw['address']['house_number']

    if 'road' in response.raw['address']['road']:
        data.loc[i, 'road'] = response.raw['address']['road']




Loop: 0/21613
Loop: 1/21613
Loop: 2/21613
Loop: 3/21613
Loop: 4/21613
Loop: 5/21613
Loop: 6/21613
Loop: 7/21613
Loop: 8/21613
Loop: 9/21613
Loop: 10/21613


KeyError: 'house_number'

In [ ]:
response = geolocator.reverse('lat, log') # vai retornar um json(que é um dicionario) que em algumas chaves tem outros json(que é um dicionario) dentro dele

# Doing Map

In [50]:
import plotly.express as px

houses = data[['id', 'lat', 'long', 'price', 'level']].copy() # temos que fazer esse copy, pois senao ele vai ficar trabalhando com o ponteiro de data e nao com uma copia de data

fig = px.scatter_mapbox( houses,
                   lat='lat',
                   lon='long',
                   color='level',
                   size='price',
                   color_continuous_scale=px.colors.cyclical.Twilight,
                   size_max=15,
                   zoom=10 )

fig.update_layout( mapbox_style='open-street-map' )
fig.update_layout( height=600, margin={ 'r': 0, 'l':0, 'b':0, 't':0 } )
fig.show()
